# Solución etapa 4 - FE + Training

In [64]:
# Utilidades para print
from utils.print_utils import tabl, headr, titl

In [65]:
# Cargar los datasets
import pandas as pd

historicos_ordenes_cleaned = pd.read_csv('../data/cleaned/historicos_ordenes_cleaned.csv')
registros_condiciones_cleaned = pd.read_csv('../data/cleaned/registros_condiciones_cleaned.csv')
caracteristicas_equipos_cleaned = pd.read_csv('../data/cleaned/caracteristicas_equipos_cleaned.csv')


## Feature eng

### Target

In [66]:
# Generar target

target_column="Fallo"

def code_target(val):
    if val == 'Correctivo':
        return 1
    else:
        return 0


historicos_ordenes_cleaned[target_column] = historicos_ordenes_cleaned["Tipo_Mantenimiento"].apply(code_target)
historicos_ordenes_cleaned.drop("Tipo_Mantenimiento", axis=1, inplace=True)

### historicos_ordenes

In [67]:
tabl(historicos_ordenes_cleaned)

      ID_Orden    ID_Equipo  Fecha         Costo_Mantenimiento    Duracion_Horas  Ubicacion      Fallo
--  ----------  -----------  ----------  ---------------------  ----------------  -----------  -------
 0           1          382  2021-01-01                   2619                 5  Ubicacion_A        1
 1           2          350  2021-01-01                   2619                 5  Ubicacion_A        1
 2           3          302  2021-01-02                   2619                 5  Ubicacion_B        1
 3           4           55  2021-01-02                   1556                 2  Ubicacion_D        1
 4           5           89  2021-01-02                   2619                 5  Ubicacion_B        1
(31866, 7)


In [68]:
historicos_ordenes_cleaned['Fecha'] = pd.to_datetime(historicos_ordenes_cleaned['Fecha'])

# historicos_ordenes_cleaned['Año'] = historicos_ordenes_cleaned['Fecha'].dt.year
# historicos_ordenes_cleaned['Mes'] = historicos_ordenes_cleaned['Fecha'].dt.month
# historicos_ordenes_cleaned['Día'] = historicos_ordenes_cleaned['Fecha'].dt.day

tabl(historicos_ordenes_cleaned)

      ID_Orden    ID_Equipo  Fecha                  Costo_Mantenimiento    Duracion_Horas  Ubicacion      Fallo
--  ----------  -----------  -------------------  ---------------------  ----------------  -----------  -------
 0           1          382  2021-01-01 00:00:00                   2619                 5  Ubicacion_A        1
 1           2          350  2021-01-01 00:00:00                   2619                 5  Ubicacion_A        1
 2           3          302  2021-01-02 00:00:00                   2619                 5  Ubicacion_B        1
 3           4           55  2021-01-02 00:00:00                   1556                 2  Ubicacion_D        1
 4           5           89  2021-01-02 00:00:00                   2619                 5  Ubicacion_B        1
(31866, 7)


### registros_condiciones

In [69]:
registros_condiciones_cleaned.drop(columns=['ID_Registro'], index=1, inplace=True)

registros_condiciones_cleaned['Fecha']=pd.to_datetime(registros_condiciones_cleaned['Fecha'])
# registros_condiciones_cleaned['Año'] = registros_condiciones_cleaned['Fecha'].dt.year
# registros_condiciones_cleaned['Mes'] = registros_condiciones_cleaned['Fecha'].dt.month
# registros_condiciones_cleaned['Día'] = registros_condiciones_cleaned['Fecha'].dt.day

tabl(registros_condiciones_cleaned)

      ID_Equipo  Fecha                  Temperatura_C    Vibracion_mm_s    Horas_Operativas
--  -----------  -------------------  ---------------  ----------------  ------------------
 0            1  2021-01-01 00:00:00          36.5871           4.43038                 579
 2          446  2021-01-01 00:00:00          47.1765           7.11696                 263
 3          107  2021-01-01 00:00:00          76.4046           2.69691                 491
 4          108  2021-01-01 00:00:00          50.7293           6.99677                 589
 5          445  2021-01-01 00:00:00          62.5769           7.93556                 519
(730499, 5)


### Unir registro + hitorico + caracteristicas

In [70]:
# Unir datasets
final_data = pd.merge(registros_condiciones_cleaned, historicos_ordenes_cleaned, on=['ID_Equipo','Fecha'], how='left')
final_data = pd.merge(final_data, caracteristicas_equipos_cleaned, on='ID_Equipo', how='inner')

tabl(final_data)

      ID_Equipo  Fecha                  Temperatura_C    Vibracion_mm_s    Horas_Operativas    ID_Orden    Costo_Mantenimiento    Duracion_Horas    Ubicacion    Fallo  Tipo_Equipo    Fabricante    Modelo      Potencia_kW    Horas_Recomendadas_Revision
--  -----------  -------------------  ---------------  ----------------  ------------------  ----------  ---------------------  ----------------  -----------  -------  -------------  ------------  --------  -------------  -----------------------------
 0            1  2021-01-01 00:00:00          36.5871           4.43038                 579         nan                    nan               nan          nan      nan  Bomba          Fabricante_A  Modelo_8            432                            657
 1          446  2021-01-01 00:00:00          47.1765           7.11696                 263         nan                    nan               nan          nan      nan  Generador      Fabricante_C  Modelo_2             95                        

In [71]:
final_data.columns

Index(['ID_Equipo', 'Fecha', 'Temperatura_C', 'Vibracion_mm_s',
       'Horas_Operativas', 'ID_Orden', 'Costo_Mantenimiento', 'Duracion_Horas',
       'Ubicacion', 'Fallo', 'Tipo_Equipo', 'Fabricante', 'Modelo',
       'Potencia_kW', 'Horas_Recomendadas_Revision'],
      dtype='object')

In [72]:
# Limpiar columnas accesorias
final_data.drop(columns=['ID_Orden','Costo_Mantenimiento', 'Duracion_Horas', 'Ubicacion'], inplace=True)

tabl(final_data)

      ID_Equipo  Fecha                  Temperatura_C    Vibracion_mm_s    Horas_Operativas    Fallo  Tipo_Equipo    Fabricante    Modelo      Potencia_kW    Horas_Recomendadas_Revision
--  -----------  -------------------  ---------------  ----------------  ------------------  -------  -------------  ------------  --------  -------------  -----------------------------
 0            1  2021-01-01 00:00:00          36.5871           4.43038                 579      nan  Bomba          Fabricante_A  Modelo_8            432                            657
 1          446  2021-01-01 00:00:00          47.1765           7.11696                 263      nan  Generador      Fabricante_C  Modelo_2             95                            561
 2          107  2021-01-01 00:00:00          76.4046           2.69691                 491      nan  Compresor      Fabricante_A  Modelo_7            195                            748
 3          108  2021-01-01 00:00:00          50.7293           6.9967

### Duplicados

In [73]:
# Eliminar duplicados

print(headr('Duplicados final_data:'),final_data.duplicated().sum())
# final_data.drop_duplicates(inplace=True)
# tabl(final_data)


Duplicados final_data:
 0


In [74]:
# Verificamos nulos
print(headr('Nulos final_data:'),final_data.isna().sum())


Nulos final_data:
 ID_Equipo                           0
Fecha                               0
Temperatura_C                       0
Vibracion_mm_s                      0
Horas_Operativas                    0
Fallo                          698870
Tipo_Equipo                         0
Fabricante                          0
Modelo                              0
Potencia_kW                         0
Horas_Recomendadas_Revision         0
dtype: int64


In [75]:
columnas_con_nulos = final_data.columns[final_data.isna().any()].tolist()
columnas_con_nulos

['Fallo']

In [76]:
# Imputamos nulos
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy="constant", fill_value=0)

final_data[columnas_con_nulos] = imputer.fit_transform(final_data[columnas_con_nulos])

In [77]:
tabl(final_data)

      ID_Equipo  Fecha                  Temperatura_C    Vibracion_mm_s    Horas_Operativas    Fallo  Tipo_Equipo    Fabricante    Modelo      Potencia_kW    Horas_Recomendadas_Revision
--  -----------  -------------------  ---------------  ----------------  ------------------  -------  -------------  ------------  --------  -------------  -----------------------------
 0            1  2021-01-01 00:00:00          36.5871           4.43038                 579        0  Bomba          Fabricante_A  Modelo_8            432                            657
 1          446  2021-01-01 00:00:00          47.1765           7.11696                 263        0  Generador      Fabricante_C  Modelo_2             95                            561
 2          107  2021-01-01 00:00:00          76.4046           2.69691                 491        0  Compresor      Fabricante_A  Modelo_7            195                            748
 3          108  2021-01-01 00:00:00          50.7293           6.9967

In [78]:
print("% fallos:",
final_data[final_data['Fallo'] == 1].shape[0] / final_data.shape[0] * 100)

% fallos: 0.9746337938735741


In [79]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730736 entries, 0 to 730735
Data columns (total 11 columns):
 #   Column                       Non-Null Count   Dtype         
---  ------                       --------------   -----         
 0   ID_Equipo                    730736 non-null  int64         
 1   Fecha                        730736 non-null  datetime64[ns]
 2   Temperatura_C                730736 non-null  float64       
 3   Vibracion_mm_s               730736 non-null  float64       
 4   Horas_Operativas             730736 non-null  int64         
 5   Fallo                        730736 non-null  float64       
 6   Tipo_Equipo                  730736 non-null  object        
 7   Fabricante                   730736 non-null  object        
 8   Modelo                       730736 non-null  object        
 9   Potencia_kW                  730736 non-null  int64         
 10  Horas_Recomendadas_Revision  730736 non-null  int64         
dtypes: datetime64[ns](1), floa

### Volvemos a explorar con columnas codificadas


In [80]:
# Extraer columnas numéricas y categóricas
from sklearn.compose import make_column_selector as selector

numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(final_data)
categorical_columns = categorical_columns_selector(final_data)

print(headr("Numerical columns"), numerical_columns)
print(headr("Categorical columns"), categorical_columns)


Numerical columns
 ['ID_Equipo', 'Fecha', 'Temperatura_C', 'Vibracion_mm_s', 'Horas_Operativas', 'Fallo', 'Potencia_kW', 'Horas_Recomendadas_Revision']

Categorical columns
 ['Tipo_Equipo', 'Fabricante', 'Modelo']


In [81]:
from sklearn.preprocessing import LabelEncoder, StandardScaler

final_data_coded = final_data.copy()
final_data_coded[categorical_columns] = final_data_coded[categorical_columns].apply(LabelEncoder().fit_transform)

# Eliminamos fecha para la evaluación
final_data_coded.drop(columns=['Fecha'], inplace=True)

tabl(final_data_coded)

      ID_Equipo    Temperatura_C    Vibracion_mm_s    Horas_Operativas    Fallo    Tipo_Equipo    Fabricante    Modelo    Potencia_kW    Horas_Recomendadas_Revision
--  -----------  ---------------  ----------------  ------------------  -------  -------------  ------------  --------  -------------  -----------------------------
 0            1          36.5871           4.43038                 579        0              0             0         8            432                            657
 1          446          47.1765           7.11696                 263        0              2             2         2             95                            561
 2          107          76.4046           2.69691                 491        0              1             0         7            195                            748
 3          108          50.7293           6.99677                 589        0              1             0         9            181                            560
 4        

In [82]:
# Evaluamos la aportación de cada columna
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif,chi2

X= final_data_coded.drop(target_column, axis=1)
y = final_data_coded[target_column]

fvalue_selector = SelectKBest(f_classif, k=2)

X_kbest = fvalue_selector.fit(X,y)

feature_scores = pd.DataFrame({"Feature": X.columns,"Score": X_kbest.scores_}).sort_values(by="Score", ascending=False)

print(headr("Feature scores"))
round(feature_scores,2)


Feature scores



,Feature,Score
3,Horas_Operativas,8006.48
8,Horas_Recomendadas_Revision,3.07
0,ID_Equipo,1.46
4,Tipo_Equipo,0.56
7,Potencia_kW,0.25
5,Fabricante,0.24
6,Modelo,0.03
2,Vibracion_mm_s,0.01
1,Temperatura_C,0.00


> Una característica tiene demasiado peso y determina toda la clasificación

In [83]:
# Eliminamos columnas sin peso

columns_to_drop = feature_scores[feature_scores['Score'] < 0.1]['Feature']
final_data.drop(columns=columns_to_drop, inplace=True)
tabl(final_data)

      ID_Equipo  Fecha                  Horas_Operativas    Fallo  Tipo_Equipo    Fabricante      Potencia_kW    Horas_Recomendadas_Revision
--  -----------  -------------------  ------------------  -------  -------------  ------------  -------------  -----------------------------
 0            1  2021-01-01 00:00:00                 579        0  Bomba          Fabricante_A            432                            657
 1          446  2021-01-01 00:00:00                 263        0  Generador      Fabricante_C             95                            561
 2          107  2021-01-01 00:00:00                 491        0  Compresor      Fabricante_A            195                            748
 3          108  2021-01-01 00:00:00                 589        0  Compresor      Fabricante_A            181                            560
 4          445  2021-01-01 00:00:00                 519        0  Compresor      Fabricante_A             92                            586
(730736, 8)


## Exportar 

In [85]:
final_data.to_csv('../data/preprocessed/preprocessed_data_v1.csv', index=False)